# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,brooks,50.5834,-111.8851,5.99,75,100,6.69,CA,2024-02-03 20:15:02
1,1,san carlos del zulia,9.0000,-71.9135,32.23,58,100,4.60,VE,2024-02-03 20:19:29
2,2,ballina,-28.8667,153.5667,24.88,85,99,3.19,AU,2024-02-03 20:19:29
3,3,avarua,-21.2078,-159.7750,28.03,78,36,2.57,CK,2024-02-03 20:12:09
4,4,albany,42.6001,-73.9662,3.36,60,64,2.24,US,2024-02-03 20:09:25


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    color='City',
    alpha=0.5,
    s='Humidity',
    scale=0.8)

# Display the map
plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_min = 20
ideal_temp_max = 30
ideal_humidity_max = 70
ideal_wind_speed_max = 10
ideal_weather_df = city_data_df.loc[(city_data_df['Max Temp']>=ideal_temp_min) &
                            (city_data_df['Max Temp']<=ideal_temp_max) &
                            (city_data_df['Humidity']<=ideal_humidity_max) &
                            (city_data_df['Wind Speed']<=ideal_wind_speed_max)]


# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
print(ideal_weather_df)

     City_ID         City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
32        32       acarau  -2.8856  -40.1200     28.90        70          78   
33        33    porbandar  21.6422   69.6093     23.15        62           0   
34        34      chonchi -42.6128  -73.8092     20.75        43          41   
51        51          wau   7.7029   27.9953     25.47        21          99   
69        69    kavaratti  10.5669   72.6420     26.73        70          32   
..       ...          ...      ...       ...       ...       ...         ...   
542      542       guiuan  11.0333  125.7247     27.56        70          70   
543      543  port isabel  26.0734  -97.2086     26.05        68           0   
547      547      mochudi -24.4167   26.1500     26.33        41           3   
553      553      namanga  -2.5433   36.7905     21.43        69         100   
556      556       belabo   4.9333   13.3000     23.96        29          56   

     Wind Speed Country                

### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

#Coordinates = ('Lat', 'Lng')
                   
hotel_df = ideal_weather_df.loc[:,['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
print(hotel_df)

            City Country      Lat       Lng  Humidity Hotel Name
32        acarau      BR  -2.8856  -40.1200        70           
33     porbandar      IN  21.6422   69.6093        62           
34       chonchi      CL -42.6128  -73.8092        43           
51           wau      SS   7.7029   27.9953        21           
69     kavaratti      IN  10.5669   72.6420        70           
..           ...     ...      ...       ...       ...        ...
542       guiuan      PH  11.0333  125.7247        70           
543  port isabel      US  26.0734  -97.2086        68           
547      mochudi      BW -24.4167   26.1500        41           
553      namanga      KE  -2.5433   36.7905        69           
556       belabo      CM   4.9333   13.3000        29           

[70 rows x 6 columns]


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 20
params = {'categories': categories,
        'limit': limit,
        'apiKey': geoapify_key
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filter = f'circle: {lon},{lat},{radius}'
    bias = f'proximity:{lon},{lat}'

    params["filters"] = filter
    params["bias"] = bias
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
acarau - nearest hotel: No hotel found
porbandar - nearest hotel: Toran Tourist Bungalow
chonchi - nearest hotel: Hotel Huildín
wau - nearest hotel: Safari Park Hotel
kavaratti - nearest hotel: Bangaram Island Beach Resort
coahuayana de hidalgo - nearest hotel: Hotel Águila Real
el porvenir - nearest hotel: Hotel Oasis Colonial
kwekwe - nearest hotel: Shamwarl Hotel
hobe sound - nearest hotel: Pirate's Cove Resort & Marina
humberto de campos - nearest hotel: Pousada Água Doce
brisas de zicatela - nearest hotel: Casa de Olas
gunnedah - nearest hotel: The Court House
sikasso - nearest hotel: Hotel Piemont
cockburn town - nearest hotel: The Salt Raker Inn
basoko - nearest hotel: No hotel found
alice springs - nearest hotel: Aurora Alice Springs
valdivia - nearest hotel: Esmeralda
mindelo - nearest hotel: Solar Windelo
araripe - nearest hotel: Hotel do Município de Campos Sales
port mathurin - nearest hotel: Escale Vacances
mitu - nearest hotel: Hotel Brasilia Vaupés


,City,Country,Lat,Lng,Humidity,Hotel Name
32,acarau,BR,-2.8856,-40.1200,70,No hotel found
33,porbandar,IN,21.6422,69.6093,62,Toran Tourist Bungalow
34,chonchi,CL,-42.6128,-73.8092,43,Hotel Huildín
51,wau,SS,7.7029,27.9953,21,Safari Park Hotel
69,kavaratti,IN,10.5669,72.6420,70,Bangaram Island Beach Resort
...,...,...,...,...,...,...
542,guiuan,PH,11.0333,125.7247,70,Addison Pension House
543,port isabel,US,26.0734,-97.2086,68,Queen Isabel Inn
547,mochudi,BW,-24.4167,26.1500,41,Rasesa Lodge
553,namanga,KE,-2.5433,36.7905,69,Namange Masaai Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    color='City',
    alpha=0.5,
    use_index=False,
    hover_cols=['Hotel Name', 'Country'],
    s='Humidity',
    scale=0.9)

# Display the map
plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)